In [1]:
import numpy as np
from keras.preprocessing import sequence
from tqdm import tqdm
import preprocessing as preproc
from utils import *

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
/opt/conda/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
path = app_3_length_15_data_path
base_path = path + val_folder

NR_TRAIN_INSTANCES = 82000
BATCH_SIZE = 2048
MAX_CAPTION_LEN = 15

In [3]:
images_precomputed_vgg_features = preproc.read_serialized_np_arr(base_path+images_vgg_4096_folder+'vgg_features.bc'
                                                                  ,nr_instances = NR_TRAIN_INSTANCES )

print(images_precomputed_vgg_features.shape)

raw_captions = preproc.get_truncated_captions_from_batch(base_path + captions_folder, batch_nr = 0,
                                                           nr_instances = NR_TRAIN_INSTANCES )

print(len(raw_captions))

unique_words = preproc.load_obj(path+general_datastruct_folder+"unique_words")
word2index = preproc.load_obj(path+general_datastruct_folder+"word2index")
index2word = preproc.load_obj(path+general_datastruct_folder+"index2word")
VOCAB_SIZE = len(unique_words)
print(len(unique_words))

(28858, 4096)
28858
7275


In [4]:
prev_indexed_captions = []
for index,raw_caption in tqdm(enumerate(raw_captions)):
    indexed_caption = [word2index[caption_word] for caption_word in raw_caption.split()]
    
    #this is where the magic happens
    indexed_caption.insert(0, indexed_caption[0])
    indexed_caption = indexed_caption[:-1]
    
    indexed_caption = sequence.pad_sequences([indexed_caption], maxlen=MAX_CAPTION_LEN,padding='post')
    indexed_np_arr = np.asarray(np.squeeze(indexed_caption))
    
    prev_indexed_captions.append(indexed_np_arr)

28858it [00:00, 35942.26it/s]


In [5]:
indexed_prev_captions_folder = "indexed-prev-captions/"

In [7]:
for index in tqdm(range(len(prev_indexed_captions)/BATCH_SIZE)):
    
    indexed_caption_list = []
     
    for elem_in_batch in range(BATCH_SIZE):
        indexed_caption = np.expand_dims(prev_indexed_captions[index*BATCH_SIZE + elem_in_batch],axis=0)
        indexed_caption_list.append(indexed_caption)
        
    indexed_captions_batch = np.vstack(indexed_caption_list)
    
    save_array(base_path + batch_folder + indexed_prev_captions_folder + 'indexed_prev_caption_'
               + str(format(index, "06")) + '_' + '.bc',
               indexed_captions_batch)

100%|██████████| 14/14 [00:00<00:00, 117.56it/s]
